<a href="https://colab.research.google.com/github/miinsss/Capstone-Project/blob/main/Random_Forest_and_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libaries

In [24]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import recall_score
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier

# Load the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load the contract_classification.csv file
contract_class_path = '/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/contracts_with_new_data_MEAN.csv'
contract_class = pd.read_csv(contract_class_path)



# Display the first few rows to verify content
print(contract_class.head())

<ipython-input-3-283dbef524b6>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  contract_class = pd.read_csv(contract_class_path)


   Unnamed: 0    ANO_SID CORPORATE_DEVISION  ORTPLZ  ORTS-NAME  \
0           0  4114028.0                VHV   42109  Wuppertal   
1           1  4114039.0                VHV   42277  Wuppertal   
2           2  4114045.0                VHV   42389  Wuppertal   
3           3  4114049.0                VGV   42277  Wuppertal   
4           4  4114055.0                VHV   42553    Velbert   

                STRASSE  SUM_INSURED CONSTRACTION_DESIGN  CONSTRUCTION_YEAR  \
0     Hans-Böckler-Str.      71081.0      NORMAL_VENTURE        1967.565648   
1       Liegnitzer Str.      55708.0      NORMAL_VENTURE        1967.565648   
2             Rascheweg      74148.0      DESIGN_CLASS_I        1967.565648   
3               Am Diek     664000.0      NORMAL_VENTURE        1967.565648   
4  Emil-Schniewind-Str.      75682.0      NORMAL_VENTURE        1967.565648   

         WFL  ... PRIOR_DAMAGES  UVV-KZ  UNDERWRITER              PARTY-ID  \
0   69.00000  ...             0       1           

In [ ]:
print(contract_class['DAMAGE'].value_counts())


na_counts = contract_class.isna().sum()

print(na_counts)

DAMAGE
0    10203688
1      170852
Name: count, dtype: int64
Unnamed: 0                      0
ANO_SID                        50
CORPORATE_DEVISION              0
ORTPLZ                          0
ORTS-NAME                      15
STRASSE                      8516
SUM_INSURED                 14832
CONSTRACTION_DESIGN       1083360
CONSTRUCTION_YEAR               0
WFL                         14996
ZONE                       877533
SF-SYSTEM                 8713940
TYPE_OF_DEDUCTIBLE              0
DRAIN_PIPE_INSURED              0
PRODUCTLINE               1583747
PRIOR_DAMAGES                   0
UVV-KZ                          0
UNDERWRITER                     0
PARTY-ID                      191
contract_year                   0
PIPE_PREMIUM_AMOUNT           164
YEAR                            0
DAMAGE                          0
DAMAGE_FLOOD_ZONE               0
DAMAGE_HEAVY_RAIN_ZONE          0
dtype: int64


# Models

## Preprocess the data

In [4]:
#Choose variables where it makes logical sense

initial_features = contract_class.loc[:, ['ANO_SID','CORPORATE_DEVISION','ORTPLZ','ZONE', 'PRODUCTLINE', 'CONSTRACTION_DESIGN','CONSTRUCTION_YEAR','WFL','ZONE','DRAIN_PIPE_INSURED', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER','DAMAGE','DAMAGE_FLOOD_ZONE','DAMAGE_HEAVY_RAIN_ZONE']]

print(initial_features['DAMAGE'].value_counts())

initial_features_clean = initial_features.dropna()

print(initial_features_clean['DAMAGE'].value_counts())

DAMAGE
0    10203688
1      170852
Name: count, dtype: int64
DAMAGE
0    7680154
1     112491
Name: count, dtype: int64


In [5]:
# Choose columns to encode to binary variables

columns_to_encode = ['CORPORATE_DEVISION','CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create Binary Variables
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)

print(features_binary.head())

     ANO_SID  ORTPLZ  CONSTRUCTION_YEAR    WFL  DRAIN_PIPE_INSURED  \
0  4114028.0   42109        1967.565648   69.0                   0   
1  4114039.0   42277        1967.565648   65.0                   0   
2  4114045.0   42389        1967.565648   75.0                   0   
4  4114055.0   42553        1967.565648  119.0                   0   
5  4114057.0   42113        1967.565648  100.0                   0   

   PRIOR_DAMAGES  UVV-KZ  DAMAGE  DAMAGE_FLOOD_ZONE  DAMAGE_HEAVY_RAIN_ZONE  \
0              0       1       0                1.0                1.727273   
1              0       1       0                1.0                1.931034   
2              0       1       0                1.0                1.962963   
4              0       1       0                1.0                1.768000   
5              0       1       0                1.0                1.824324   

   ...  ZONE_4  ZONE_4.0  ZONE_5  ZONE_5.0  ZONE_6  ZONE_6.0  ZONE_7  \
0  ...   False     False   False

## Random Forest

In [7]:
# Define feature and target

X = features_binary.drop(columns=['ANO_SID','DAMAGE' ])
y = features_binary['DAMAGE']

# Split into test and training

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [10]:
# Initializing Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)

# Training the classifier
rf_classifier.fit(X_train, y_train)

# Making probability predictions
y_pred_proba = rf_classifier.predict_proba(X_test)

# Evaluating the model

# Converting probabilities to class predictions
y_pred = np.argmax(y_pred_proba, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generating confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Displaying confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Extracting TP, FP, TN, FN from the confusion matrix
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]

# Printing TP, FP, TN, FN
print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)

# Classification report
print(classification_report(y_test, y_pred))
# With undersampling accuracy gets worse, but we detect more TRUE Positives. Still way more false positive

Accuracy: 0.9862601209217153
Confusion Matrix:
[[1532264    3724]
 [  17690    4851]]
True Positives: 4851
False Positives: 3724
True Negatives: 1532264
False Negatives: 17690
              precision    recall  f1-score   support

           0       0.99      1.00      0.99   1535988
           1       0.57      0.22      0.31     22541

    accuracy                           0.99   1558529
   macro avg       0.78      0.61      0.65   1558529
weighted avg       0.98      0.99      0.98   1558529



In [ ]:
# Feature-importnace analysis
feature_importance = rf_classifier.feature_importances_

# Display results
print("Feature Importance:")
for i, feature in enumerate(X_train.columns):
    print(f"{feature}: {feature_importance[i]}")

Feature Importance:
ORTPLZ: 0.16774241717665753
CONSTRUCTION_YEAR: 0.109547584905663
WFL: 0.15623796903442802
DRAIN_PIPE_INSURED: 0.0
PRIOR_DAMAGES: 0.004829930303817164
UVV-KZ: 0.01632314332811708
DAMAGE_FLOOD_ZONE: 0.059112125055463746
DAMAGE_HEAVY_RAIN_ZONE: 0.13988391013083562
YEAR: 0.10239182515558566
CORPORATE_DEVISION_VHV: 0.14627492930636185
CONSTRACTION_DESIGN_CARAVAN_MOTORHOME: 6.366908560959066e-06
CONSTRACTION_DESIGN_DESIGN_CLASS_I: 0.000303016183868281
CONSTRACTION_DESIGN_DESIGN_CLASS_II: 0.0
CONSTRACTION_DESIGN_DESIGN_CLASS_III: 0.0
CONSTRACTION_DESIGN_DESIGN_CLASS_IV: 0.0
CONSTRACTION_DESIGN_DESIGN_CLASS_V: 6.344638606475288e-07
CONSTRACTION_DESIGN_NORMAL_VENTURE: 0.0011960665849346588
CONSTRACTION_DESIGN_PREDOMINANTLY_WOODEN_ROOF: 0.00015296769852954644
CONSTRACTION_DESIGN_PREFAB_HOUSE: 0.0008995705202636431
CONSTRACTION_DESIGN_PREFAB_HOUSE_I: 0.0
PRODUCTLINE_Kompakt: 0.013900746169091604
PRODUCTLINE_Plus: 0.0026051411443416537
PRODUCTLINE_Premium: 0.0022136500619829584

## Random Forest with  Resampling

In [11]:
# Define feature and target
X = features_binary.drop(columns=['ANO_SID','DAMAGE'])
y = features_binary['DAMAGE']

# Split into test and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize RandomUnderSampler
under_sampler = RandomUnderSampler(random_state=42)

# Resample the training data
X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)

# Print the shape of resampled data
print('Original dataset shape:', y_train.value_counts())
print('Resampled dataset shape:', y_resampled.value_counts())

KeyboardInterrupt: 

In [ ]:
# Initializing Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42)

# Training the classifier
rf_classifier.fit(X_resampled, y_resampled)

# Making probability predictions
y_pred_proba = rf_classifier.predict_proba(X_test)

In [ ]:
# Evaluating the model

# Converting probabilities to class predictions
y_pred = np.argmax(y_pred_proba, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generating confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Displaying confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Extracting TP, FP, TN, FN from the confusion matrix
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]

# Printing TP, FP, TN, FN
print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)

# Classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.7983528057546572
Confusion Matrix:
[[1227367  308621]
 [   5652   16889]]
True Positives: 16889
False Positives: 308621
True Negatives: 1227367
False Negatives: 5652
              precision    recall  f1-score   support

           0       1.00      0.80      0.89   1535988
           1       0.05      0.75      0.10     22541

    accuracy                           0.80   1558529
   macro avg       0.52      0.77      0.49   1558529
weighted avg       0.98      0.80      0.88   1558529



## Random Forest with Rolling window and SMOTE

In [12]:
#Choose variables where it makes logical sense

initial_features = contract_class.loc[:, ['ANO_SID','CORPORATE_DEVISION','ORTPLZ','ZONE', 'PRODUCTLINE', 'CONSTRACTION_DESIGN','CONSTRUCTION_YEAR','WFL','ZONE','DRAIN_PIPE_INSURED', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER','DAMAGE','DAMAGE_FLOOD_ZONE','DAMAGE_HEAVY_RAIN_ZONE','YEAR']]

print(initial_features['DAMAGE'].value_counts())

initial_features_clean = initial_features.dropna()

print(initial_features_clean['DAMAGE'].value_counts())

DAMAGE
0    10203688
1      170852
Name: count, dtype: int64
DAMAGE
0    7680154
1     112491
Name: count, dtype: int64


In [29]:
# Choose columns to encode to binary variables

columns_to_encode = ['CORPORATE_DEVISION','CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create Binary Variables
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)


In [14]:
# Define feature and target

X = features_binary.drop(columns=['DAMAGE', 'ANO_SID'])
y = features_binary['DAMAGE']



In [16]:
# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'] == train_year]
    y_train = y[features_binary['YEAR'] == train_year]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

    ## Initialize the SMOTE resampler
    smote = SMOTE(random_state=42)

    # Resample the training data
    X_resampled, y_resampled =  smote.fit_resample(X_train, y_train)

    # Initialize and train Random Forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)
    rf_classifier.fit(X_resampled, y_resampled)

    # Make predictions (probabilities)
    y_prob = rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

    # Accumulate true and predicted values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    # Create a DataFrame for the probabilities
    probabilities_df = pd.DataFrame({'Probability': y_prob})

    # Combine the probabilities with the original test data for the year
    combined_df = initial_features_clean[initial_features_clean['YEAR'] == test_year].copy()
    combined_df['Probability'] = probabilities_df['Probability'].values

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(f'probabilities_{test_year}.csv', index=False)

    # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)

# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")


# Combine all the CSV files into a single DataFrame
combined_all_years = pd.concat([pd.read_csv(f'probabilities_{year}.csv') for year in years[1:]])

# Save the combined DataFrame to a CSV file
combined_all_years.to_csv('combined_probabilities_all_years.csv', index=False)

print("Combined probabilities for all years:")
print(combined_all_years.head())



  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    692820
           1       0.27      0.30      0.28      9020

    accuracy                           0.98    701840
   macro avg       0.63      0.64      0.64    701840
weighted avg       0.98      0.98      0.98    701840

Confusion Matrix for year 2015:
[[685546   7274]
 [  6317   2703]]
True Positives: 2703
False Positives: 7274
True Negatives: 685546
False Negatives: 6317


 10%|█         | 1/10 [00:32<04:52, 32.48s/it]

Classification report for year 2016:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    719837
           1       0.30      0.29      0.30     10068

    accuracy                           0.98    729905
   macro avg       0.65      0.64      0.64    729905
weighted avg       0.98      0.98      0.98    729905

Confusion Matrix for year 2016:
[[713008   6829]
 [  7122   2946]]
True Positives: 2946
False Positives: 6829
True Negatives: 713008
False Negatives: 7122


 20%|██        | 2/10 [01:07<04:29, 33.73s/it]

...

Classification report for year 2023:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    716445
           1       0.16      0.17      0.17     10366

    accuracy                           0.98    726811
   macro avg       0.58      0.58      0.58    726811
weighted avg       0.98      0.98      0.98    726811

Confusion Matrix for year 2023:
[[707442   9003]
 [  8599   1767]]
True Positives: 1767
False Positives: 9003
True Negatives: 707442
False Negatives: 8599


 90%|█████████ | 9/10 [05:29<00:38, 38.60s/it]

Classification report for year 2024:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    721440
           1       0.03      0.15      0.05      1900

    accuracy                           0.99    723340
   macro avg       0.51      0.57      0.52    723340
weighted avg       1.00      0.99      0.99    723340

Confusion Matrix for year 2024:
[[712728   8712]
 [  1616    284]]
True Positives: 284
False Positives: 8712
True Negatives: 712728
False Negatives: 1616


100%|██████████| 10/10 [06:09<00:00, 36.96s/it]


Combined Confusion Matrix:
[[6934534   81714]
 [  80910   22385]]
Combined True Positives: 22385
Combined False Positives: 81714
Combined True Negatives: 6934534
Combined False Negatives: 80910
Combined Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99   7016248
           1       0.22      0.22      0.22    103295

    accuracy                           0.98   7119543
   macro avg       0.60      0.60      0.60   7119543
weighted avg       0.98      0.98      0.98   7119543

Year: 2015, Recall Score: 0.29966740576496675
Year: 2016, Recall Score: 0.2926102502979738
Year: 2017, Recall Score: 0.276778413736713
Year: 2018, Recall Score: 0.19019104647847163
Year: 2019, Recall Score: 0.19085764040052242
Year: 2020, Recall Score: 0.2060211188496967
Year: 2021, Recall Score: 0.18795081404135466
Year: 2022, Recall Score: 0.18764806796830324
Year: 2023, Recall Score: 0.17046112290179433
Year: 2024, Recall Score: 0.149473684

In [19]:
#Cont number of entries
num_entries = len(combined_all_years)
print(num_entries)

# Total number of NaN values
total_na = combined_all_years.isna().sum().sum()
print(total_na)

#Show range of prob
min_prob = combined_all_years['Probability'].min()
max_prob = combined_all_years['Probability'].max()
print(f"Range of probabilities in the combined DataFrame: {min_prob} to {max_prob}")



7119543
0
Range of probabilities in the combined DataFrame: 0.0 to 1.0


## Random Forest with roling window and k fold cross validation

In [23]:
# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []

# Initialize the Stratified K-Fold Cross Validation
n_splits = 5  # You can adjust the number of splits
skf = StratifiedKFold(n_splits=n_splits)

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'] == train_year]
    y_train = y[features_binary['YEAR'] == train_year]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

    # Perform Stratified K-Fold Cross Validation within the training year
    for train_index, val_index in skf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

         # Fit your model here on X_train_fold and y_train_fold
        model = RandomForestClassifier(n_estimators=10, random_state=42)
        model.fit(X_train_fold, y_train_fold)

        # Validate your model here on X_val_fold and y_val_fold
        y_val_pred = model.predict(X_val_fold)
        fold_recall = recall_score(y_val_fold, y_val_pred)
        recall_scores.append(fold_recall)

    # Fit the model on the entire training set after cross-validation
    model.fit(X_train, y_train)

    # Make predictions (probabilities)
    y_prob = rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

    # Accumulate true and predicted values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    # Create a DataFrame for the probabilities
    probabilities_df = pd.DataFrame({'Probability': y_prob})

    # Combine the probabilities with the original test data for the year
    combined_df = initial_features_clean[initial_features_clean['YEAR'] == test_year].copy()
    combined_df['Probability'] = probabilities_df['Probability'].values

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(f'probabilities_{test_year}.csv', index=False)

    # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)

# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    692820
           1       0.29      0.19      0.23      9020

    accuracy                           0.98    701840
   macro avg       0.64      0.59      0.61    701840
weighted avg       0.98      0.98      0.98    701840

Confusion Matrix for year 2015:
[[688660   4160]
 [  7282   1738]]
True Positives: 1738
False Positives: 4160
True Negatives: 688660
False Negatives: 7282


 10%|█         | 1/10 [00:58<08:47, 58.59s/it]

Classification report for year 2016:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    719837
           1       0.27      0.19      0.23     10068

    accuracy                           0.98    729905
   macro avg       0.63      0.59      0.61    729905
weighted avg       0.98      0.98      0.98    729905

Confusion Matrix for year 2016:
[[714663   5174]
 [  8111   1957]]
True Positives: 1957
False Positives: 5174
True Negatives: 714663
False Negatives: 8111


 20%|██        | 2/10 [01:59<08:01, 60.16s/it]

...

 90%|█████████ | 9/10 [09:54<01:09, 69.39s/it]

Classification report for year 2024:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    721440
           1       0.03      0.15      0.05      1900

    accuracy                           0.99    723340
   macro avg       0.51      0.57      0.52    723340
weighted avg       1.00      0.99      0.99    723340

Confusion Matrix for year 2024:
[[712728   8712]
 [  1616    284]]
True Positives: 284
False Positives: 8712
True Negatives: 712728
False Negatives: 1616


100%|██████████| 10/10 [11:03<00:00, 66.40s/it]


Combined Confusion Matrix:
[[6959884   56364]
 [  79088   24207]]
Combined True Positives: 24207
Combined False Positives: 56364
Combined True Negatives: 6959884
Combined False Negatives: 79088
Combined Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99   7016248
           1       0.30      0.23      0.26    103295

    accuracy                           0.98   7119543
   macro avg       0.64      0.61      0.63   7119543
weighted avg       0.98      0.98      0.98   7119543



TypeError: cannot unpack non-iterable numpy.float64 object

## Balanced Random Forest with Roling window

In [28]:
# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'] == train_year]
    y_train = y[features_binary['YEAR'] == train_year]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]



    # Initialize and train Random Forest classifier
    rf_classifier = BalancedRandomForestClassifier(n_estimators=50, random_state=42)
    rf_classifier.fit(X_train, y_train)

    # Make predictions (probabilities)
    y_prob = rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

    # Accumulate true and predicted values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)



    # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)

# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py

Classification report for year 2015:


 10%|█         | 1/10 [00:14<02:10, 14.46s/it]

              precision    recall  f1-score   support

           0       1.00      0.78      0.87    692820
           1       0.04      0.78      0.08      9020

    accuracy                           0.78    701840
   macro avg       0.52      0.78      0.48    701840
weighted avg       0.98      0.78      0.86    701840

Confusion Matrix for year 2015:
[[538730 154090]
 [  1954   7066]]
True Positives: 7066
False Positives: 154090
True Negatives: 538730
False Negatives: 1954


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.para

...

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.para

Classification report for year 2024:


100%|██████████| 10/10 [02:40<00:00, 16.06s/it]

              precision    recall  f1-score   support

           0       1.00      0.70      0.82    721440
           1       0.01      0.81      0.01      1900

    accuracy                           0.70    723340
   macro avg       0.50      0.76      0.42    723340
weighted avg       1.00      0.70      0.82    723340

Confusion Matrix for year 2024:
[[506613 214827]
 [   357   1543]]
True Positives: 1543
False Positives: 214827
True Negatives: 506613
False Negatives: 357


Combined Confusion Matrix:
[[5264190 1752058]
 [  23245   80050]]
Combined True Positives: 80050
Combined False Positives: 1752058
Combined True Negatives: 5264190
Combined False Negatives: 23245
Combined Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86   7016248
           1       0.04      0.77      0.08    103295

    accuracy                           0.75   7119543
   macro avg       0.52      0.76      0.47   7119543
weighted avg       0.98      0.75      0.84   7119543

Year: 2015, Recall Score: 0.783370288248337
Year: 2016, Recall Score: 0.7741358760429082
Year: 2017, Recall Score: 0.7522485690923958
Year: 2018, Recall Score: 0.736146754110826
Year: 2019, Recall Score: 0.7749238136700044
Year: 2020, Recall Score: 0.7722609151501535
Year: 2021, Recall Score: 0.7770831902177646
Year: 2022, Recall Score: 0.8000980312066007
Year: 2023, Recall Score: 0.7933629172294038
Year: 2024, Recall Score: 0.81210526315789

## Random Forest with expanding window and tuning

In [ ]:

# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [10, 20],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []

# Loop over each year for rolling window estimation
for i in tqdm(range(1, len(years))):
    test_year = years[i]
    train_years = years[:i]  # All years up to the year before test_year

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'].isin(train_years)]
    y_train = y[features_binary['YEAR'].isin(train_years)]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

    # Initialize the Random Forest classifier
    rf_classifier = RandomForestClassifier(random_state=42)


    # Perform Grid Search with cross-validation
    grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                               cv=3, n_jobs=-1, verbose=2, scoring='recall')
    grid_search.fit(X_train, y_train)

    # Get the best model from Grid Search
    best_rf_classifier = grid_search.best_estimator_

    # Make predictions (probabilities)
    y_prob = best_rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the best parameters for the current year
    print(f"Best parameters for year {test_year}: {grid_search.best_params_}")

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)
    # Accumulate true and predicted values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

     # Create a DataFrame for the probabilities
    probabilities_df = pd.DataFrame({'Probability': y_prob})

    # Combine the probabilities with the original test data for the year
    combined_df = initial_features_clean[initial_features_clean['YEAR'] == test_year].copy()
    combined_df['Probability'] = probabilities_df['Probability'].values

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(f'probabilities_{test_year}.csv', index=False)

       # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)


# Combine all the CSV files into a single DataFrame
combined_all_years = pd.concat([pd.read_csv(f'probabilities_{year}.csv') for year in years[1:]])

# Save the combined DataFrame to a CSV file
combined_all_years.to_csv('combined_probabilities_all_years.csv', index=False)

print("Combined probabilities for all years:")
print(combined_all_years.head())

  0%|          | 0/10 [00:00<?, ?it/s]

Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best parameters for year 2015: {'max_depth': 30, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 20}
Classification report for year 2015:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    692820
           1       0.86      0.25      0.39      9020

    accuracy                           0.99    701840
   macro avg       0.93      0.63      0.69    701840
weighted avg       0.99      0.99      0.99    701840

Confusion Matrix for year 2015:
[[692447    373]
 [  6734   2286]]
True Positives: 2286
False Positives: 373
True Negatives: 692447
False Negatives: 6734


 10%|█         | 1/10 [04:05<36:52, 245.84s/it]

Fitting 3 folds for each of 54 candidates, totalling 162 fits


NameError: name 'combined_all_years_df' is not defined

# Naive Bayes

In [30]:
# Define initial features
initial_features = contract_class.loc[:, ['ANO_SID','CORPORATE_DEVISION', 'SUM_INSURED', 'CONSTRACTION_DESIGN', 'CONSTRUCTION_YEAR', 'WFL', 'ZONE', 'DRAIN_PIPE_INSURED', 'PRODUCTLINE', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER', 'DAMAGE','DAMAGE_FLOOD_ZONE','DAMAGE_HEAVY_RAIN_ZONE', 'YEAR']]
initial_features_clean = initial_features.dropna()

# Create binary variables using one-hot encoding
columns_to_encode = ['CORPORATE_DEVISION', 'CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create binary variables using one-hot encoding
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)

X = features_binary.drop(columns=['ANO_SID','DAMAGE'])
y = features_binary['DAMAGE']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Separate continuous and discrete features
continuous_features = ['SUM_INSURED', 'CONSTRUCTION_YEAR','DAMAGE_FLOOD_ZONE','DAMAGE_HEAVY_RAIN_ZONE' ]
# The discrete features need to be adjusted after one-hot encoding
discrete_features = list(set(X.columns) - set(continuous_features))

## Naive Bayes with rolling window

In [34]:
# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'] == train_year]
    y_train = y[features_binary['YEAR'] == train_year]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

# Train GaussianNB on continuous features
    gnb = GaussianNB()
    gnb.fit(X_train[continuous_features], y_train)

# Train MultinomialNB on discrete features
    mnb = MultinomialNB()
    mnb.fit(X_train[discrete_features], y_train)

    # Get predicted probabilities for continuous and discrete features
    gnb_prob = gnb.predict_proba(X_test[continuous_features])[:, 1]
    mnb_prob = mnb.predict_proba(X_test[discrete_features])[:, 1]

    # Combine probabilities (you can average them, use weighted sum, etc.)
    combined_prob = (gnb_prob + mnb_prob) / 2

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (combined_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))



    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

     # Accumulate true and predicted values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)
# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:


 10%|█         | 1/10 [00:01<00:13,  1.54s/it]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    692820
           1       0.41      0.12      0.19      9020

    accuracy                           0.99    701840
   macro avg       0.70      0.56      0.59    701840
weighted avg       0.98      0.99      0.98    701840

Confusion Matrix for year 2015:
[[691247   1573]
 [  7927   1093]]
True Positives: 1093
False Positives: 1573
True Negatives: 691247
False Negatives: 7927
Classification report for year 2016:


 20%|██        | 2/10 [00:03<00:12,  1.52s/it]

...

 90%|█████████ | 9/10 [00:13<00:01,  1.53s/it]

              precision    recall  f1-score   support

           0       0.99      0.97      0.98    716445
           1       0.04      0.07      0.05     10366

    accuracy                           0.96    726811
   macro avg       0.51      0.52      0.51    726811
weighted avg       0.97      0.96      0.97    726811

Confusion Matrix for year 2023:
[[696188  20257]
 [  9608    758]]
True Positives: 758
False Positives: 20257
True Negatives: 696188
False Negatives: 9608
Classification report for year 2024:


100%|██████████| 10/10 [00:15<00:00,  1.52s/it]

              precision    recall  f1-score   support

           0       1.00      0.96      0.98    721440
           1       0.01      0.09      0.01      1900

    accuracy                           0.96    723340
   macro avg       0.50      0.52      0.49    723340
weighted avg       0.99      0.96      0.98    723340

Confusion Matrix for year 2024:
[[691680  29760]
 [  1731    169]]
True Positives: 169
False Positives: 29760
True Negatives: 691680
False Negatives: 1731


Combined Confusion Matrix:
[[6925687   90561]
 [  97093    6202]]
Combined True Positives: 6202
Combined False Positives: 90561
Combined True Negatives: 6925687
Combined False Negatives: 97093
Combined Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99   7016248
           1       0.06      0.06      0.06    103295

    accuracy                           0.97   7119543
   macro avg       0.53      0.52      0.52   7119543
weighted avg       0.97      0.97      0.97   7119543

Year: 2015, Recall Score: 0.12117516629711751
Year: 2016, Recall Score: 0.10885975367500993
Year: 2017, Recall Score: 0.09750613246116108
Year: 2018, Recall Score: 0.023857047809143616
Year: 2019, Recall Score: 0.025337396604266434
Year: 2020, Recall Score: 0.03197783269677226
Year: 2021, Recall Score: 0.03778251013258226
Year: 2022, Recall Score: 0.05007760803855894
Year: 2023, Recall Score: 0.07312367354813815
Year: 2024, Recall Score: 0.0889

Naive BAyes with roling window and smote

In [35]:
# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'] == train_year]
    y_train = y[features_binary['YEAR'] == train_year]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

# Apply SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Train GaussianNB on continuous features
    gnb = GaussianNB()
    gnb.fit(X_train_resampled[continuous_features], y_train_resampled)

    # Train MultinomialNB on discrete features
    mnb = MultinomialNB()
    mnb.fit(X_train_resampled[discrete_features], y_train_resampled)

    # Get predicted probabilities for continuous and discrete features
    gnb_prob = gnb.predict_proba(X_test[continuous_features])[:, 1]
    mnb_prob = mnb.predict_proba(X_test[discrete_features])[:, 1]

    # Combine probabilities (you can average them, use weighted sum, etc.)
    combined_prob = (gnb_prob + mnb_prob) / 2

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (combined_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))



    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

     # Accumulate true and predicted values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)
# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:


 10%|█         | 1/10 [00:03<00:27,  3.05s/it]

              precision    recall  f1-score   support

           0       0.99      0.98      0.98    692820
           1       0.08      0.14      0.10      9020

    accuracy                           0.97    701840
   macro avg       0.53      0.56      0.54    701840
weighted avg       0.98      0.97      0.97    701840

Confusion Matrix for year 2015:
[[677579  15241]
 [  7737   1283]]
True Positives: 1283
False Positives: 15241
True Negatives: 677579
False Negatives: 7737
Classification report for year 2016:


 20%|██        | 2/10 [00:06<00:24,  3.05s/it]

...

 90%|█████████ | 9/10 [00:28<00:03,  3.22s/it]

              precision    recall  f1-score   support

           0       0.99      0.90      0.94    716445
           1       0.02      0.13      0.03     10366

    accuracy                           0.89    726811
   macro avg       0.50      0.52      0.49    726811
weighted avg       0.97      0.89      0.93    726811

Confusion Matrix for year 2023:
[[644135  72310]
 [  8977   1389]]
True Positives: 1389
False Positives: 72310
True Negatives: 644135
False Negatives: 8977
Classification report for year 2024:


100%|██████████| 10/10 [00:31<00:00,  3.18s/it]

              precision    recall  f1-score   support

           0       1.00      0.87      0.93    721440
           1       0.00      0.18      0.01      1900

    accuracy                           0.87    723340
   macro avg       0.50      0.52      0.47    723340
weighted avg       0.99      0.87      0.93    723340

Confusion Matrix for year 2024:
[[625655  95785]
 [  1563    337]]
True Positives: 337
False Positives: 95785
True Negatives: 625655
False Negatives: 1563


Combined Confusion Matrix:
[[6617597  398651]
 [  92845   10450]]
Combined True Positives: 10450
Combined False Positives: 398651
Combined True Negatives: 6617597
Combined False Negatives: 92845
Combined Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.94      0.96   7016248
           1       0.03      0.10      0.04    103295

    accuracy                           0.93   7119543
   macro avg       0.51      0.52      0.50   7119543
weighted avg       0.97      0.93      0.95   7119543

Year: 2015, Recall Score: 0.14223946784922395
Year: 2016, Recall Score: 0.12584425903853794
Year: 2017, Recall Score: 0.11201962387571546
Year: 2018, Recall Score: 0.05075563159395495
Year: 2019, Recall Score: 0.06190683500217675
Year: 2020, Recall Score: 0.08192915449711675
Year: 2021, Recall Score: 0.09500583911520231
Year: 2022, Recall Score: 0.11077526345886773
Year: 2023, Recall Score: 0.13399575535404207
Year: 2024, Recall Score: 0.1773

## Naive Bayes with expanding window

In [ ]:
# Define parameter grids for GaussianNB and MultinomialNB
gnb_param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

mnb_param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0]
}

# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []

# Loop over each year for rolling window estimation
for i in tqdm(range(1, len(years))):
    test_year = years[i]
    train_years = years[:i]  # All years up to the year before test_year

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'].isin(train_years)]
    y_train = y[features_binary['YEAR'].isin(train_years)]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

    # Initialize GaussianNB and perform Grid Search
    gnb = GaussianNB()
    gnb_grid_search = GridSearchCV(estimator=gnb, param_grid=gnb_param_grid, cv=3, n_jobs=-1, scoring='recall')
    gnb_grid_search.fit(X_train[continuous_features], y_train)
    best_gnb = gnb_grid_search.best_estimator_

    # Initialize MultinomialNB and perform Grid Search
    mnb = MultinomialNB()
    mnb_grid_search = GridSearchCV(estimator=mnb, param_grid=mnb_param_grid, cv=3, n_jobs=-1, scoring='recall')
    mnb_grid_search.fit(X_train[discrete_features], y_train)
    best_mnb = mnb_grid_search.best_estimator_

    # Get predicted probabilities for continuous and discrete features
    gnb_prob = best_gnb.predict_proba(X_test[continuous_features])[:, 1]
    mnb_prob = best_mnb.predict_proba(X_test[discrete_features])[:, 1]

    # Combine probabilities (you can average them, use weighted sum, etc.)
    combined_prob = (gnb_prob + mnb_prob) / 2

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (combined_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the best parameters for the current year
    print(f"Best parameters for GaussianNB in year {test_year}: {gnb_grid_search.best_params_}")
    print(f"Best parameters for MultinomialNB in year {test_year}: {mnb_grid_search.best_params_}")

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]

Best parameters for GaussianNB in year 2015: {'var_smoothing': 1e-09}
Best parameters for MultinomialNB in year 2015: {'alpha': 0.1}
Classification report for year 2015:


 10%|█         | 1/10 [00:08<01:17,  8.58s/it]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    775271
           1       0.42      0.12      0.19      9736

    accuracy                           0.99    785007
   macro avg       0.70      0.56      0.59    785007
weighted avg       0.98      0.99      0.98    785007

Confusion Matrix for year 2015:
[[773620   1651]
 [  8539   1197]]
True Positives: 1197
False Positives: 1651
True Negatives: 773620
False Negatives: 8539
Best parameters for GaussianNB in year 2016: {'var_smoothing': 1e-09}
Best parameters for MultinomialNB in year 2016: {'alpha': 2.0}
Classification report for year 2016:


 20%|██        | 2/10 [00:18<01:15,  9.39s/it]

...

              precision    recall  f1-score   support

           0       0.98      0.99      0.99    797092
           1       0.06      0.04      0.05     13298

    accuracy                           0.97    810390
   macro avg       0.52      0.51      0.52    810390
weighted avg       0.97      0.97      0.97    810390

Confusion Matrix for year 2022:
[[789534   7558]
 [ 12814    484]]
True Positives: 484
False Positives: 7558
True Negatives: 789534
False Negatives: 12814
Best parameters for GaussianNB in year 2023: {'var_smoothing': 1e-09}
Best parameters for MultinomialNB in year 2023: {'alpha': 0.1}
Classification report for year 2023:


 90%|█████████ | 9/10 [03:07<00:28, 28.48s/it]

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    800711
           1       0.05      0.05      0.05     11149

    accuracy                           0.97    811860
   macro avg       0.52      0.52      0.52    811860
weighted avg       0.97      0.97      0.97    811860

Confusion Matrix for year 2023:
[[790632  10079]
 [ 10566    583]]
True Positives: 583
False Positives: 10079
True Negatives: 790632
False Negatives: 10566
Best parameters for GaussianNB in year 2024: {'var_smoothing': 1e-09}
Best parameters for MultinomialNB in year 2024: {'alpha': 0.1}
Classification report for year 2024:


100%|██████████| 10/10 [03:48<00:00, 22.86s/it]

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    806672
           1       0.01      0.05      0.01      2049

    accuracy                           0.98    808721
   macro avg       0.50      0.52      0.50    808721
weighted avg       1.00      0.98      0.99    808721

Confusion Matrix for year 2024:
[[793852  12820]
 [  1944    105]]
True Positives: 105
False Positives: 12820
True Negatives: 793852
False Negatives: 1944
Year: 2015, Recall Score: 0.12294576828266228
Year: 2016, Recall Score: 0.10920018492834027
Year: 2017, Recall Score: 0.09618510373840711
Year: 2018, Recall Score: 0.029283216783216784
Year: 2019, Recall Score: 0.02676282051282051
Year: 2020, Recall Score: 0.027824151363383415
Year: 2021, Recall Score: 0.02599873176918199
Year: 2022, Recall Score: 0.036396450594074294
Year: 2023, Recall Score: 0.052291685352946454
Year: 2024, Recall Score: 0.05124450951683748
